In [2]:
import pandas as pd

import pandas as pd
import nltk
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [4]:
df = pd.read_csv("data/data-v0.csv")
print(len(df))
df.head()

2347


,conversation_text,conversation_tag
0,"Agent: Hi, this is Kevin from Holland and Barr...",store closing hours
1,"Agent: Good morning, Holland and Barrett custo...",repeat purchases
2,"Agent: Hi, thank you for contacting Holland an...",supplement advice
3,"Agent: Hello, Holland & Barrett customer suppo...",staff interaction
4,Agent: Thank you for calling Holland & Barrett...,hair products


In [5]:
df.conversation_tag.value_counts()

conversation_tag
discount appreciation      42
gratitude                  42
friendly staff             39
order cancellation         36
product recommendations    34
                           ..
voucher issues             18
discontinued products      16
intimidating behavior      15
misleading information     14
inconsistent policies      13
Name: count, Length: 87, dtype: int64

In [36]:
for x in df.sample(20).iterrows():
    print(x[1].conversation_tag)
    print(x[1].conversation_text)

order cancellation
Agent: Good morning, Holland and Barrett customer service. How may I help you?
Visitor: Hi, I ordered some protein bars, but I need to cancel the order now.
Agent: Certainly. Can you provide your order number?
Visitor: Yes, it's [REDACTED].
Agent: Thank you. One moment... Your order is still being processed. I'll cancel it for you and process the refund. It should appear in your account within 3-5 business days.
Visitor: That's great. Thanks for your help.
Agent: You're welcome. Have a nice day.
Visitor: You too.

promotions and offers
Agent: Hello, thank you for contacting Holland and Barrett. How can I help you today?
Visitor: Hi, I saw an offer for 'buy one get one for a penny' but it's not working on the products I selected. Can you check what's going on?
Agent: Certainly. Could you please provide the product names or SKUs?
Visitor: Sure, it's for the Omega-3 Fish Oil capsules and the Vitamin D tablets.
Agent: Thank you. Let me check if these products are include

# Most frequent words

In [8]:
def get_top_words(df, top_n=10, min_word_length=3):
    stop_words = set(stopwords.words('english') + ["agent", "visitor", "thank"])
    
    tag_top_words = {}
    
    # Group by conversation tag
    for tag, group in df.groupby('conversation_tag'):
        all_text = ' '.join(group['conversation_text'])
        
        words = word_tokenize(all_text.lower())
        
        filtered_words = [
            word for word in words 
            if (word.isalnum() and 
                word not in stop_words and 
                len(word) >= min_word_length)
        ]
        
        word_freq = Counter(filtered_words)
        
        tag_top_words[tag] = word_freq.most_common(top_n)
    
    return tag_top_words

In [9]:
top_words = get_top_words(df)
    
# Print results
for tag, words in top_words.items():
    print(f"Top words for tag '{tag}':")
    for word, count in words:
        print(f"  {word}: {count}")


Top words for tag 'accurate product information':
  refund: 52
  website: 46
  order: 43
  would: 40
  product: 39
  replacement: 37
  number: 33
  please: 33
  great: 33
  day: 32
Top words for tag 'address issues':
  address: 95
  order: 76
  number: 49
  delivered: 45
  help: 28
  provide: 28
  welcome: 28
  redacted: 28
  sorry: 27
  day: 27
Top words for tag 'allergy-sensitive products':
  refund: 59
  number: 47
  order: 44
  product: 41
  please: 35
  redacted: 35
  would: 35
  assist: 29
  help: 29
  thanks: 28
Top words for tag 'app assistance':
  app: 54
  try: 42
  let: 38
  thanks: 36
  please: 35
  assist: 34
  holland: 32
  barrett: 32
  great: 30
  hear: 29
Top words for tag 'app/website usability issues':
  please: 61
  could: 35
  help: 34
  holland: 32
  barrett: 32
  great: 32
  website: 31
  let: 30
  email: 30
  try: 29
Top words for tag 'authentic products':
  number: 91
  batch: 70
  holland: 46
  barrett: 46
  genuine: 44
  please: 41
  help: 40
  welcome: 38
  

# Topic Modeling

In [2]:
from bertopic import BERTopic
import pandas as pd
import numpy as np
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv("data/data-v0.csv")
print(len(df))
df.head()

2347


,conversation_text,conversation_tag
0,"Agent: Hi, this is Kevin from Holland and Barr...",store closing hours
1,"Agent: Good morning, Holland and Barrett custo...",repeat purchases
2,"Agent: Hi, thank you for contacting Holland an...",supplement advice
3,"Agent: Hello, Holland & Barrett customer suppo...",staff interaction
4,Agent: Thank you for calling Holland & Barrett...,hair products


In [4]:
topic_model = BERTopic()


In [5]:
%%time
topics, probs = topic_model.fit_transform(df.conversation_text)
topics = np.array(topics)

df_topics = topic_model.get_topic_info()
len(topics), len(df_topics)

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


CPU times: user 2min 18s, sys: 20.2 s, total: 2min 39s
Wall time: 43.2 s


(2347, 65)

In [6]:
topics_str = [df_topics[df_topics.Topic == x].iloc[0].Name for x in topics]
len(topics_str)

2347

In [8]:
with pd.option_context('display.max_rows', None):
    print(df_topics)

    Topic  Count                                               Name
0      -1    526                               -1_you_for_agent_the
1       0     98                 0_feedback_wanted_share_experience
2       1     90                  1_damaged_order_items_replacement
3       2     81                2_price_prices_discounts_promotions
4       3     80                        3_protein_vegan_powder_have
5       4     76                             4_order_delay_for_your
6       5     76                       5_vitamin_iron_energy_levels
7       6     71                           6_delay_order_next_agent
8       7     66                 7_bottle_order_replacement_damaged
9       8     46                   8_ready_collection_order_collect
10      9     43                       9_address_delivered_it_order
11     10     41                10_staff_feedback_experience_wanted
12     11     39                  11_product_refund_bought_reaction
13     12     37                        12_tea_d

In [34]:

with pd.option_context('display.max_rows', None):
    print(df.conversation_tag.value_counts())

conversation_tag
discount appreciation           42
gratitude                       42
friendly staff                  39
order cancellation              36
product recommendations         34
staff interaction               33
store display                   33
vegetarian and vegan options    33
faulty/damaged products         32
positive recommendations        32
impulse buying                  32
health products                 32
product condition               31
online ordering                 31
pricing issues                  31
accurate product information    31
promotion and offers            31
follow-up issues                31
order status updates            31
nutritional information         31
store stock availability        30
unresponsive support            30
product selection               30
order not received              30
supplement advice               30
product effectiveness           30
expiration date issues          29
staff knowledge                 29
val

## Checking the relation between provided tags and the generated topics

In [10]:
df_temp = pd.DataFrame(zip(df.conversation_tag, topics_str), columns=["tag", "topic"])
print(len(df_temp))
df_temp.head()

2347


,tag,topic
0,store closing hours,15_closed_store_home_early
1,repeat purchases,-1_you_for_agent_the
2,supplement advice,58_anxiety_ashwagandha_stress_magnesium
3,staff interaction,44_staff_manager_service_will
4,hair products,37_omega3_capsules_fish_oil


In [14]:
for x in df_temp.groupby("tag"):
    print(x[0], len(x[1]))
    df_tag_group = x[1]
    df_tag_group = df_tag_group[
        df_tag_group.topic != df_topics[df_topics.Topic == -1].iloc[0].Name
    ]
    df_tag_group_counts = df_tag_group.topic.value_counts()
    print(df_tag_group_counts[df_tag_group_counts >= 5])
    print("\n")

accurate product information 31
Series([], Name: count, dtype: int64)


address issues 28
topic
9_address_delivered_it_order    28
Name: count, dtype: int64


allergy-sensitive products 26
topic
18_glutenfree_refund_replacement_pasta    11
11_product_refund_bought_reaction          5
Name: count, dtype: int64


app assistance 29
topic
62_app_using_try_crashing    6
Name: count, dtype: int64


app/website usability issues 28
Series([], Name: count, dtype: int64)


authentic products 29
topic
29_batch_genuine_number_from    21
Name: count, dtype: int64


brand loyalty 19
topic
26_products_skin_skincare_sensitive    5
Name: count, dtype: int64


click and collect 29
topic
8_ready_collection_order_collect    14
Name: count, dtype: int64


communication issues 20
topic
4_order_delay_for_your    16
Name: count, dtype: int64


complaint resolution 27
topic
51_complaint_reference_soon_this    10
Name: count, dtype: int64


convenient location 22
topic
12_tea_delivery_from_herbal    12
Name: co

From the above list, we can see that the provided tags do not form clear topic clusters, instead a lot of the tags have 2-3 topics which tell us that the tags can be a but more granular

In [21]:
topic_index = 10
print("Name:", df_topics[df_topics.Topic == topic_index].Name.iloc[0], end="\n\n")
print(*df.conversation_text[pd.Series(topics) == topic_index].tolist(), sep="\n\n")

Name: 10_staff_feedback_experience_wanted

Agent: Hello, Holland & Barrett customer service, how can I help?
Visitor: Hi, I was at your store in Leeds and needed help choosing a vitamin supplement. The staff member seemed uninterested and didn't offer much assistance.
Agent: I apologize for that experience. We aim to provide knowledgeable and helpful service at all times. Can you tell me more about the interaction?
Visitor: Sure, I asked for a vegetarian option, but the staff just handed me a random bottle without checking if it met my needs.
Agent: That’s certainly not the level of service we expect from our staff. I will escalate this issue to ensure better training. Thank you for letting us know. Can I assist you with anything else?
Visitor: No, that's all. Thank you.
Agent: Thank you for your feedback. Have a great day.
Visitor: You too.


Agent: Hello, this is Holland & Barrett customer support. How can I help you?
Visitor: Hi, I was at your Lymington store recently and wanted to 